<a href="https://colab.research.google.com/github/lblogan14/Python_Deep_Learning/blob/master/ch1_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction to PyTorch
PyTorch is an open source python deep learning framework, developed primarily by Facebook that has been gaining momentum recently. It provides the **Graphics Processing Unit (GPU)**, an accelerated multidimensional array (or **tensor**) operation, and computational graphs.

Begin with a toy example:

1. Prepare Iris dataset

In [0]:
import pandas as pd

In [0]:
dataset = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                      names=['sepal_length',
                             'sepal_width',
                             'petal_length',
                             'petal_width',
                             'species'])

In [0]:
dataset['species'] = pd.Categorical(dataset['species']).codes
dataset = dataset.sample(frac=1, random_state=1234)

train_input = dataset.values[:120, :4]
train_target = dataset.values[:120, 4]

test_input = dataset.values[120:, :4]
test_target = dataset.values[120:, 4]

This code downloads the Iris dataset and loads into the pandas DataFrame, and then shuffles the DataFrame rows and split the code into numpy arrays.

2. Define a simple neural network (make sure the PyTorch is installed)

In [0]:
import torch

In [0]:
torch.manual_seed(1234)

hidden_units = 5

net = torch.nn.Sequential(torch.nn.Linear(4, hidden_units),
                          torch.nn.ReLU(),
                          torch.nn.Linear(hidden_units, 3))

This defines a feedforward network with one hidden layer with five units, a ReLU activation function, and an output layer with three units.

Note that the target data is *one-hot encoded*. This means that each class of the flower will be represented as an array (`Iris Setosa = [1, 0, 0], Iris
Versicolour = [0, 1, 0], and Iris Virginica = [0, 0, 1]`), and one
element of the array will be the target for one unit of the output layer.

3. Define an optimizer and a loss function

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), 
                            lr=0.1,
                            momentum=0.9)

4. Train the network

In [7]:
epochs = 50

for epoch in range(epochs):
  inputs = torch.autograd.Variable(torch.Tensor(train_input).float())
  targets = torch.autograd.Variable(torch.Tensor(train_target).long())
  
  optimizer.zero_grad()
  out = net(inputs)
  loss = criterion(out, targets)
  loss.backward()
  optimizer.step()
  
  if epoch == 0 or (epoch+1)% 10 == 0:
    print('Epoch %d Loss: %.4f' % (epoch+1, loss.item()))

Epoch 1 Loss: 1.2181
Epoch 10 Loss: 0.6745
Epoch 20 Loss: 0.2447
Epoch 30 Loss: 0.1397
Epoch 40 Loss: 0.1001
Epoch 50 Loss: 0.0855


Here, the training takes 50 epochs.

A few things need to be mentioned here:
1. Create the **torch variable** that are **input** and **target** 
2. Zero the gradients or the optimizer to prevent accumulation from the previous iterations
3. Propagate the loss value back through the network (*backpropagation*)

5. Test the accuracy

In [0]:
import numpy as np

In [9]:
inputs = torch.autograd.Variable(torch.Tensor(test_input).float())
targets = torch.autograd.Variable(torch.Tensor(test_target).long())

optimizer.zero_grad()
out = net(inputs)
_, predicted = torch.max(out.data, 1)

error_count = test_target.size - np.count_nonzero((targets==predicted).numpy())
print('Errors: %d; Accuracy: %d%%' % (error_count, 
                                      100*torch.sum(targets==predicted)/test_target.size))

Errors: 0; Accuracy: 100%
